# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [130]:
# Use this cell to write your code for Task 1
import pandas as pd
task1 = pd.read_csv("house_sales.csv")
city_values = task1['city'].value_counts(dropna=False)
missing_city = city_values['--']
print(missing_city)

73


# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [131]:
# Use this cell to write your code for Task 2
# city = '--'
# months_listed = null
# house_type needs cleaning (change "Det.", "Semi", and "Terr." to its corresponding)
import numpy as np

clean_data = task1.copy()
clean_data.loc[clean_data['city'] == '--', ['city']] = "Unknown"
mean_months_listed = round(clean_data['months_listed'].mean(), 1)
clean_data['months_listed'].fillna(mean_months_listed, inplace=True)
clean_data.loc[clean_data['house_type'] == 'Det.', ['house_type']] = "Detached"
clean_data.loc[clean_data['house_type'] == 'Semi', ['house_type']] = "Semi-detached"
clean_data.loc[clean_data['house_type'] == 'Terr.', ['house_type']] = "Terraced"
clean_data['area'] = clean_data['area'].str.strip(' sq.m.')
clean_data

,house_id,city,sale_price,sale_date,months_listed,bedrooms,house_type,area
0,1217792,Silvertown,55943,2021-09-12,5.4,2,Semi-detached,107.8
1,1900913,Silvertown,384677,2021-01-17,6.3,5,Detached,498.8
2,1174927,Riverford,281707,2021-11-10,6.9,6,Detached,542.5
3,1773666,Silvertown,373251,2020-04-13,6.1,6,Detached,528.4
4,1258487,Silvertown,328885,2020-09-24,8.7,5,Detached,477.1
...,...,...,...,...,...,...,...,...
1495,1123892,Riverford,198661,2022-02-17,7.3,5,Detached,432.2
1496,1327295,Poppleton,358304,2020-10-10,6.2,6,Detached,599.8
1497,1058161,Riverford,176612,2022-11-01,6.8,4,Detached,359.1
1498,1822037,Teasdale,197827,2021-04-03,5.4,3,Detached,253.7


In [132]:
clean_data['city'] = clean_data['city'].astype('category')
clean_data['sale_date'] = pd.to_datetime(clean_data['sale_date'])
clean_data['house_type'] = pd.Categorical(clean_data['house_type'], categories=["Terraced", "Semi-detached", "Detached"], ordered=True)
clean_data['area'] = clean_data['area'].astype('float64')
clean_data.dtypes

house_id                  int64
city                   category
sale_price                int64
sale_date        datetime64[ns]
months_listed           float64
bedrooms                  int64
house_type             category
area                    float64
dtype: object

In [133]:
clean_data.head()

,house_id,city,sale_price,sale_date,months_listed,bedrooms,house_type,area
0,1217792,Silvertown,55943,2021-09-12,5.4,2,Semi-detached,107.8
1,1900913,Silvertown,384677,2021-01-17,6.3,5,Detached,498.8
2,1174927,Riverford,281707,2021-11-10,6.9,6,Detached,542.5
3,1773666,Silvertown,373251,2020-04-13,6.1,6,Detached,528.4
4,1258487,Silvertown,328885,2020-09-24,8.7,5,Detached,477.1


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [134]:
# Use this cell to write your code for Task 3
task3 = pd.read_csv('house_sales.csv')
price_by_rooms = task3.groupby('bedrooms')['sale_price'].agg(['mean', 'var'])
price_by_rooms.reset_index(inplace=True)
price_by_rooms.rename(columns={"mean": "avg_price", "var": "var_price"}, inplace=True)
price_by_rooms = price_by_rooms.round(1)
price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [135]:
# Use this cell to write your code for Task 4
from sklearn.preprocessing import StandardScaler

train_data = pd.read_csv("train.csv")
validation_data = pd.read_csv("validation.csv")
                              
train_data['city'] = train_data['city'].astype('category')
train_data['sale_date'] = pd.to_datetime(train_data['sale_date'])
validation_data['sale_date'] = pd.to_datetime(validation_data['sale_date'])

# Extract features from sale_date
train_data['sale_year'] = train_data['sale_date'].dt.year
train_data['sale_month'] = train_data['sale_date'].dt.month
train_data['sale_day'] = train_data['sale_date'].dt.day
train_data['sale_dayofweek'] = train_data['sale_date'].dt.dayofweek  # 0=Monday, 6=Sunday
train_data['sale_weekofyear'] = train_data['sale_date'].dt.isocalendar().week  # Week number

validation_data['sale_year'] = validation_data['sale_date'].dt.year
validation_data['sale_month'] = validation_data['sale_date'].dt.month
validation_data['sale_day'] = validation_data['sale_date'].dt.day
validation_data['sale_dayofweek'] = validation_data['sale_date'].dt.dayofweek
validation_data['sale_weekofyear'] = validation_data['sale_date'].dt.isocalendar().week

# Find the earliest sale date in training data
first_sale_date = train_data['sale_date'].min()

# Compute days since first sale
train_data['days_since_first_sale'] = (train_data['sale_date'] - first_sale_date).dt.days
validation_data['days_since_first_sale'] = (validation_data['sale_date'] - first_sale_date).dt.days

train_data = train_data.drop(columns=['sale_date'])
validation_data = validation_data.drop(columns=['sale_date'])

train_data['house_type'] = pd.Categorical(train_data['house_type'], categories=["Terraced", "Semi-detached", "Detached"], ordered=True)

validation_data['city'] = validation_data['city'].astype('category')
validation_data['house_type'] = pd.Categorical(validation_data['house_type'], categories=["Terraced", "Semi-detached", "Detached"], ordered=True)

train_data.dtypes

house_id                    int64
city                     category
sale_price                  int64
months_listed             float64
bedrooms                    int64
house_type               category
area                      float64
sale_year                   int64
sale_month                  int64
sale_day                    int64
sale_dayofweek              int64
sale_weekofyear            UInt32
days_since_first_sale       int64
dtype: object

In [136]:
validation_data.head()

,house_id,city,months_listed,bedrooms,house_type,area,sale_year,sale_month,sale_day,sale_dayofweek,sale_weekofyear,days_since_first_sale
0,1331375,Teasdale,7.7,3,Terraced,209.7,2022,5,17,1,20,867
1,1630115,Teasdale,6.5,4,Detached,390.6,2020,6,30,1,27,181
2,1645745,Silvertown,7.4,6,Detached,556.8,2020,9,2,2,36,245
3,1336775,Silvertown,8.8,3,Semi-detached,208.3,2021,10,3,6,39,641
4,1888274,Silvertown,5.7,4,Detached,389.2,2022,6,13,0,24,894


In [137]:
from sklearn.preprocessing import OrdinalEncoder

train_data = pd.get_dummies(train_data, columns=['city'], drop_first=True)
validation_data = pd.get_dummies(validation_data, columns=['city'], drop_first=True)

encoder = OrdinalEncoder(categories=[["Terraced", "Semi-detached", "Detached"]])
train_data['house_type'] = encoder.fit_transform(train_data[['house_type']])
validation_data['house_type'] = encoder.transform(validation_data[['house_type']])
validation_data.head()

,house_id,months_listed,bedrooms,house_type,area,sale_year,sale_month,sale_day,sale_dayofweek,sale_weekofyear,days_since_first_sale,city_Riverford,city_Silvertown,city_Teasdale
0,1331375,7.7,3,0.0,209.7,2022,5,17,1,20,867,0,0,1
1,1630115,6.5,4,2.0,390.6,2020,6,30,1,27,181,0,0,1
2,1645745,7.4,6,2.0,556.8,2020,9,2,2,36,245,0,1,0
3,1336775,8.8,3,1.0,208.3,2021,10,3,6,39,641,0,1,0
4,1888274,5.7,4,2.0,389.2,2022,6,13,0,24,894,0,1,0


In [138]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

num_features = ['months_listed', 'area', 'days_since_first_sale']
transformer = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_features)
])

transformed_train = transformer.fit_transform(train_data)
transformed_val = transformer.transform(validation_data)

train_data['months_listed'] = transformed_train[:,0]
train_data['area'] = transformed_train[:,1]
validation_data['months_listed'] = transformed_val[:,0]
validation_data['area'] = transformed_val[:,1]
validation_data.head()

,house_id,months_listed,bedrooms,house_type,area,sale_year,sale_month,sale_day,sale_dayofweek,sale_weekofyear,days_since_first_sale,city_Riverford,city_Silvertown,city_Teasdale
0,1331375,0.878782,3,0.0,-0.971408,2022,5,17,1,20,867,0,0,1
1,1630115,0.275988,4,2.0,0.284986,2020,6,30,1,27,181,0,0,1
2,1645745,0.728084,6,2.0,1.439284,2020,9,2,2,36,245,0,1,0
3,1336775,1.431344,3,1.0,-0.981131,2021,10,3,6,39,641,0,1,0
4,1888274,-0.125875,4,2.0,0.275262,2022,6,13,0,24,894,0,1,0


In [139]:
X_train = train_data.drop(columns=['house_id', 'sale_price'])
y_train = train_data['sale_price']
X_val = validation_data.drop(columns=['house_id'])

In [140]:
print(X_train.head())
print(y_train.head())

   months_listed  bedrooms  ...  city_Silvertown  city_Teasdale
0       0.527152         6  ...                0              1
1       1.079714         6  ...                1              0
2      -0.276574         5  ...                1              0
3       0.225755         5  ...                1              0
4      -0.829135         4  ...                0              0

[5 rows x 13 columns]
0    401869
1    372387
2    325473
3    349469
4    199995
Name: sale_price, dtype: int64


In [141]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

model = GradientBoostingRegressor()
parameters = {"n_estimators": [350, 500, 700, 1000],
              "learning_rate": [0.025, 0.05, 0.1],
              "min_samples_leaf": [0.06, 0.08, 0.10],
              "max_depth": [6, 8, 10]}
grid = GridSearchCV(model, param_grid=parameters, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=5)
grid.fit(X_train, y_train)
print(grid.best_params_)
print("Best RMSE is: " + str(-grid.best_score_))

{'learning_rate': 0.025, 'max_depth': 6, 'min_samples_leaf': 0.06, 'n_estimators': 350}
Best RMSE is: 14932.416466566563


In [142]:
best_model = grid.best_estimator_
y_pred = best_model.predict(X_val)
base_result = pd.DataFrame({"house_id": validation_data['house_id'],
                            "price": y_pred})
base_result.head()

,house_id,price
0,1331375,79584.153894
1,1630115,307308.508773
2,1645745,390098.210999
3,1336775,110230.177125
4,1888274,274126.803752


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [143]:
# Use this cell to write your code for Task 5
from sklearn.ensemble import RandomForestRegressor

model2 = RandomForestRegressor(bootstrap=True, oob_score=True, random_state=49, criterion="friedman_mse")
parameters2 = {"n_estimators": [15, 20, 25, 30],
              "max_depth": [3, 4, 5],
              "min_samples_leaf": [0.03, 0.04, 0.05]}
grid2 = GridSearchCV(model2, param_grid=parameters2, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=5)
grid2.fit(X_train, y_train)
print(grid2.best_params_)
print("Best RMSE is: " + str(-grid2.best_score_))

{'max_depth': 5, 'min_samples_leaf': 0.03, 'n_estimators': 30}
Best RMSE is: 24887.121611261362


In [144]:
best_model2 = grid2.best_estimator_
y_pred2 = best_model2.predict(X_val)
compare_result = pd.DataFrame({"house_id": validation_data["house_id"],
                               "price": y_pred2})
compare_result.head()

,house_id,price
0,1331375,79401.075847
1,1630115,293068.061848
2,1645745,396979.864521
3,1336775,101031.035788
4,1888274,268098.625702
